![title](img/this-is-fine-spark.jpeg)

## 🔥 Spark fires 🔥 - unnecessary Python UDFs

In this scenario, we will demonstrate the performance impact of using unnecessary Python UDFs when we could use SQL or Dataframe API calls instead. 

_What is an unnecessary UDF I hear you ask?_ An unnecessary UDF is a UDF you wrote because you were too lazy to work out how to do it in the Spark Dataframe DSL. 😂 - possibly a little harsh but you get the idea. There are a lot things we can do in the Spark DSLs to avoid having to write Python UDFs. 

Using these Python UDFs will cause data to be (de)serialized to and from the underlying Python worker executing our UDFs, which is horrible for performance. Note only that but because the UDF is a black-box, as far as Spark is c, Catalyst, views it as a black-box optimisation opportunities 

### Bootstrapping

In [ ]:
import os

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = (
    SparkSession
    .builder.master("spark://spark:7077")
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
    # .config("spark.eventLog.enabled", "true")
    # .config("spark.eventLog.dir", "/data/tmp/spark-events")
    .appName("unnecessary-udfs")
    .getOrCreate()
)

spark.version

### Let's prep our data

We are going to borrow some test data from the excellent _Spark, The Definitive Guide_ Git repo. 

In [ ]:
# !mkdir -p /data/bike-data
# !wget https://raw.githubusercontent.com/udacity/data-analyst/master/projects/bike_sharing/201508_station_data.csv -P /data/bike-data
# !wget https://raw.githubusercontent.com/udacity/data-analyst/master/projects/bike_sharing/201508_trip_data.csv -P /data/bike-data

In [ ]:
!ls /data/bike-data

In [ ]:
output_data_path = '/data/bike-data-dates-extracted'

Next we will create some input data with two file splits for demonstration purposes.

In [ ]:
from pyspark.sql.functions import array, explode, sequence, lit

df = spark.read.option("header", True).csv("/data/bike-data/201508_trip_data.csv")
df = df.repartition(12)

# let's do a little bit of cheeky dataset expansion using the explode function
df = df.withColumn("array_column", array(sequence(lit(1), lit(2000)))) \
       .withColumn("exploded_array", explode("array_column"))

### Now let's do some data processing

For this scenario we are only interested in the data from a single partition, _start_terminal_, which we select in our filter/where clause.

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import udf, col, to_date

from pyspark.sql.types import DateType

@udf(DateType())
def convert_date(date_str: str) -> DateType:
  """Converts a date string in the format '8/19/2013' to a DateType."""

  import datetime

  date_obj = datetime.datetime.strptime(date_str.split(' ')[0], '%m/%d/%Y')
  return date_obj.date()

def process_data_with_udf(df: DataFrame) -> None:
    with_dates_df = df \
        .withColumn('start_date', convert_date(col('Start Date'))) \
        .withColumn('end_date', convert_date(col('End Date')))
    
    with_dates_df.write.mode('overwrite').parquet(output_data_path)

In [ ]:
%%time

process_data_with_udf(df)

### Putting the fire out  🔥🔥🔥 🚒 🚒 🚒 🧯🧯🧯

I get a runtime of ~50 seconds for this, which isn't terrible but it could be better. We can avoid all the overheads of talking with the Python worker by using the PySpark Dataframe DSL instead.

Let's **restart the kernel** and run again using the processing code below.

In [ ]:
def process_data_without_udf(df: DataFrame) -> None:
    with_dates_df = df \
        .withColumn('start_date', to_date(col("Start Date"), "mm/dd/yyyy HH:mm")) \
        .withColumn('end_date', to_date(col("End Date"), "mm/dd/yyyy HH:mm"))
    
    with_dates_df.write.mode('overwrite').parquet(output_data_path)

In [ ]:
%%time

process_data_without_udf(df)

### ... result  🌟🌟🌟

Whoop! With this change, my runtime is now ~30 seconds, so a **40% reduction in runtime**, nice. 👌

So what can we say? Well, firstly UDFs are great when you need them. But always favour the Spark DSL and Dataframe operations over UDFs where you can, as they will perform much better.

### Apache Arrow to the rescue 🎯

Note, if we really have to use a UDF we can always use Apache Arrow for our UDFs as discussed in the [official Spark docs](https://spark.apache.org/docs/latest/api/python/user_guide/sql/arrow_pandas.html#arrow-python-udfs), which will in many circumstances improve performance.

*Let's see what mileage we get in our case, as always let's restart the kernel and go again with the code below ...*

In [ ]:
@udf(DateType())
def convert_date_arrow(date_str: str, useArrow=True) -> DateType:
  """Converts a date string in the format '8/19/2013' to a DateType."""

  import datetime

  date_obj = datetime.datetime.strptime(date_str.split(' ')[0], '%m/%d/%Y')
  return date_obj.date()

def process_data_with_arrow_udf(df: DataFrame) -> None:
    with_dates_df = df \
        .withColumn('start_date', convert_date_arrow(col('Start Date'))) \
        .withColumn('end_date', convert_date_arrow(col('End Date')))
    
    with_dates_df.write.mode('overwrite').parquet(output_data_path)

In [ ]:
%%time

process_data_with_arrow_udf(df)

### Wrapping up

Nice, so my runtime is down by about 5 seconds, so a **10% reduction in runtime**. 👌

Whilst this is a good improvement it underlines that, where possible, we are a lot better off without any Python UDFs at all.

In [ ]:
# spark.stop()